In [17]:
import sys,os
sys.path.append('..')

# Cell 1
import sys, os, importlib, warnings, torch, pandas as pd

# 1) 把项目根目录加入到 sys.path（根据你的目录结构适当调整）
# 假设你当前在 .../V0.0.2/tutorial/ 下运行，将上一级加入路径：
proj_root = os.path.abspath(os.path.join(os.getcwd(), ".."))
if proj_root not in sys.path:
    sys.path.append(proj_root)

print("[Path] proj_root =", proj_root)
print("[Torch] version:", torch.__version__)
print("[Torch] cuda.is_available():", torch.cuda.is_available())
if torch.cuda.is_available():
    print("[Torch] device_count:", torch.cuda.device_count())
    print("[Torch] current_device:", torch.cuda.current_device())


[Path] proj_root = /root/20251010/V0.0.2
[Torch] version: 2.1.2+cu118
[Torch] cuda.is_available(): True
[Torch] device_count: 1
[Torch] current_device: 0


In [ ]:
import pandas as pd
import numpy as np
import plotly.graph_objects as go

from utils.tools import get_setting
from exp.trainer_forecasting import Trainer_Forecasting
from utils.arguments import get_args
from utils.print_args import print_args
from models.ablation_variants import build_variant

# Cell 2
from utils.arguments import get_args
from utils.print_args import print_args
from utils.tools import get_setting
from exp.trainer_forecasting import Trainer_Forecasting

from models import ablation_variants
importlib.reload(ablation_variants)
from models.ablation_variants import build_variant

print("[Import] ablation_variants loaded and reloaded.")


[Import] ablation_variants loaded and reloaded.


In [20]:
df_raw = pd.read_csv('new_ETTh1.csv')

In [21]:
def inspect_model(model):
    # 检查是否含 RevIN
    has_revin = getattr(getattr(model, 'backbone', None), 'revin', None) is not None
    print("[Check] RevIN:", has_revin)

    # 检查是否有 MoE 模块
    has_moe = any('moe' in n.lower() for n, _ in model.named_modules())
    print("[Check] MoE modules present:", has_moe)

    # 检查 dilation 配置
    dils = []
    for n, m in model.named_modules():
        if hasattr(m, 'attn') and hasattr(m.attn, 'dilation'):
            dils.append(getattr(m.attn, 'dilation'))
    print("[Check] found dilations:", dils[:5], "... total:", len(dils))


In [ ]:
import os, time, warnings
import torch
import pandas as pd

from utils.arguments import get_args
from utils.print_args import print_args
from utils.tools import get_setting
from exp.trainer_forecasting import Trainer_Forecasting
from models.ablation_variants import build_variant


# ---------------------------
# 命中率计算
# ---------------------------
def compute_hit_rates(full_pred: torch.Tensor, full_true: torch.Tensor, tolerances=(0.10, 0.05, 0.03)):
    if full_true.is_cuda: full_true = full_true.cpu()
    if full_pred.is_cuda: full_pred = full_pred.cpu()

    num_total = full_true.numel()
    if num_total == 0:
        return {"num_total": 0}

    abs_err = torch.abs(full_pred - full_true)
    out = {"num_total": int(num_total)}
    for tol in tolerances:
        hits = torch.sum(abs_err <= tol).item()
        out[f"hits@±{tol}"] = int(hits)
        out[f"hit_rate@±{tol}"] = 100.0 * hits / num_total
    return out


# ---------------------------
# 尝试把外部模型传入 Trainer；若不支持则回退
# ---------------------------
def _maybe_pass_model_to_trainer(args, df_raw, model_override=None):
    try:
        return Trainer_Forecasting(args, df_raw, model_override=model_override)
    except TypeError:
        exp = Trainer_Forecasting(args, df_raw)
        if model_override is not None:
            exp.model = model_override
            # 若你的 Trainer 有这个方法，替换模型后最好重建优化器/调度器
            if hasattr(exp, "build_optimizer"):
                exp.build_optimizer()
        return exp

import random, numpy as np, torch
# ---------------------------
# 主流程：跑所有变体
# ---------------------------
def run_all_variants(df_raw, variants=None, tolerances=(0.10, 0.05, 0.03), save_csv="ablation_results_summary.csv"):
    
    
    def set_global_seed(seed: int):
        random.seed(seed)
        np.random.seed(seed)
        torch.manual_seed(seed)
        torch.cuda.manual_seed_all(seed)
        torch.backends.cudnn.deterministic = True
        torch.backends.cudnn.benchmark = False

    if variants is None:
        variants = ['Full', 'No-RevIN', 'No-Calibration', 'No-Dilation', 'No-MoE', 'Pooling-Mean']

    # === 用与你单模型脚本一致的一套参数 ===
    parser = get_args()
    args = parser.parse_args([
        '--task_name', 'forecast',
        '--seq_len', '96',
        '--label_len', '0',
        '--pred_len', '12',
        '--use_gpu', 'True',
        '--num_workers', '0',
        '--features', 'MS',
        '--inverse',
        '--d_model','512',

        '--model', 'AD_MoE',

        # 先用 df_raw 估一个；稍后会用真实 feature_dim 回填
        '--enc_in', str(df_raw.shape[-1] - 1),
        '--dec_in', str(df_raw.shape[-1] - 1),
        '--c_out',  '1',

        '--d_ff', '4',
        '--e_layers', '3',
        '--d_layers', '3',
        '--down_sampling_layers', '3',
        '--down_sampling_window', '2',
        '--down_sampling_method', 'avg',

        '--ffn_ratio', '1',
        '--patch_size', '16',
        '--patch_stride', '8',
        '--num_blocks', '1',
        '--large_size', '51',
        '--small_size', '5',
        '--dims', '1',
        '--use_multi_scale', 'False',
        '--dropout', '0.05',
        '--head_dropout', '0.0',

        '--train_epochs', '300',
        '--batch_size', '64',
        '--learning_rate', '1e-4',
        '--optimizer', 'AdamW',
        '--lradj', 'CosineAnnealingLR',
        '--loss', 'L1',

        '--train_ratio', '0.8',
        '--val_ratio', '0.1',
        '--test_ratio', '0.1',
        '--patience', '10',
        '--weight_decay', '0.001',
        '--lr_decay_min_lr', '1e-4',
        '--lr_decay_factor', '0.9',
    ])
    print_args(args)

    # 设备兜底（Trainer 最好也实现了统一 .to(device)）
    dev = torch.device('cuda' if getattr(args, 'use_gpu', False) and torch.cuda.is_available() else 'cpu')

    rows = []
    os.makedirs(os.path.dirname(save_csv) or ".", exist_ok=True)

    for variant in variants:
        print("\n" + "="*80)
        print(f"[{time.strftime('%H:%M:%S')}] Running variant: {variant}")
        print("="*80)

        # 1) 先构建一个 exp（model_override=None），以便拿到真实特征维
        setting = get_setting(args)
        exp = _maybe_pass_model_to_trainer(args, df_raw, model_override=None)

        # ——兜底把模型搬到 device，防止 embed 等在线性层还在 CPU
        if hasattr(exp, 'model') and exp.model is not None:
            exp.model = exp.model.to(dev)
        exp.device = dev

        # 2) 从 exp 或 df_raw 得到真实 feature_dim（推荐你的 Dataset 提供 train_data.feature_dim）
        feature_dim = getattr(getattr(exp, "train_data", None), "feature_dim", None)
        if feature_dim is None:
            feature_dim = df_raw.shape[-1] - 1  # 兜底：最后一列为目标

        # 3) 回填 enc/dec/c_out
        args.enc_in = feature_dim
        args.dec_in = feature_dim
        if args.features == 'MS':
            args.c_out = 1
        print(f"[Args Sync] enc_in/dec_in -> {args.enc_in}, c_out -> {args.c_out}")

        # 4) 基于“已回填”的 args 构建变体模型
        try:
            model_override = build_variant(variant, args)
        except Exception as e:
            warnings.warn(f"[Warn] build_variant 失败：{e}，将使用 Trainer 内部模型")
            model_override = None

        # 5) 用外部模型重建 Trainer（或替换并重建优化器），并统一设备
        try:
            exp = Trainer_Forecasting(args, df_raw, model_override=model_override)
        except TypeError:
            exp = Trainer_Forecasting(args, df_raw)
            if model_override is not None:
                exp.model = model_override
                if hasattr(exp, "build_optimizer"):
                    exp.build_optimizer()

        if hasattr(exp, 'model') and exp.model is not None:
            exp.model = exp.model.to(dev)
        exp.device = dev
        inspect_model(exp.model)

        # 6) 训练 & 测试
        model, history = exp.train(setting)
        preds, trues, full_pred, full_true, metrics, base_dir, svg_path = exp.test(setting)

        # 7) 命中率
        hit_stats = compute_hit_rates(full_pred, full_true, tolerances=tolerances)

        # 8) 汇总一行
        row = {
            "variant": variant,
            "num_total": hit_stats.get("num_total", 0),
            "MAE": float(metrics.get("mae", metrics.get("MAE", float("nan")))),
            "RMSE": float(metrics.get("rmse", metrics.get("RMSE", float("nan")))),
            "MAPE": float(metrics.get("mape", metrics.get("MAPE", float("nan")))),
        }
        for k, v in hit_stats.items():
            if k != "num_total":
                row[k] = v
        rows.append(row)

        # 9) 摘要打印
        print(f"\n[Summary] {variant}")
        print(f"  MAE={row['MAE']:.4f}, RMSE={row['RMSE']:.4f}, MAPE={row['MAPE']:.4f}")
        for tol in tolerances:
            print(f"  Hit@±{tol}: {row.get(f'hit_rate@±{tol}', float('nan')):.2f}% "
                  f"({row.get(f'hits@±{tol}', 0)}/{row['num_total']})")

    # 10) 保存 CSV
    df_out = pd.DataFrame(rows)
    df_out.to_csv(save_csv, index=False, encoding="utf-8-sig")
    print(f"\n✅ Ablation hit-rate summary saved: {os.path.abspath(save_csv)}")
    return df_out


In [26]:
import random, numpy as np, torch
import pandas as pd

def set_global_seed(seed: int):
    """固定所有随机性，保证不同变体间可比性"""
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False


def run_all_variants_multi_seed(
    df_raw,
    variants=None,
    seeds=(0, 1, 2, 3, 4),
    tolerances=(0.10, 0.05, 0.03),
    save_prefix="multi_seed_results",
):
    """
    多随机种子版本的消融实验
    - 会对每个 variant 运行多次（每个 seed）
    - 输出 mean ± std 的表格
    """

    if variants is None:
        variants = ['Full', 'No-Dilation', 'No-MoE', 'Pooling-Mean']
    

    all_rows = []

    for variant in variants:
        print("\n" + "#"*100)
        print(f"🌱 Running variant: {variant}")
        print("#"*100)

        metrics_per_seed = []

        for seed in seeds:
            print(f"\n[Seed = {seed}] =====================================")
            set_global_seed(seed)

            # 调用你已有的单次函数
            df_result = run_all_variants(
                df_raw=df_raw,
                variants=[variant],
                tolerances=tolerances,
                save_csv=f"{save_prefix}_{variant}_seed{seed}.csv"
            )

            metrics_per_seed.append(df_result.iloc[0])  # 每次 run 的单行结果

        # 聚合统计
        dfm = pd.DataFrame(metrics_per_seed)
        mean_std = {
            "variant": variant,
        }
        for col in ['MAE', 'RMSE', 'MAPE', 'hit_rate@±0.03', 'hit_rate@±0.05', 'hit_rate@±0.1']:
            if col in dfm.columns:
                mean_std[col] = f"{dfm[col].mean():.6f} ± {dfm[col].std():.6f}"

        all_rows.append(mean_std)

    df_summary = pd.DataFrame(all_rows)
    save_path = f"{save_prefix}_{variant}_seed{seed}.csv"

    df_summary.to_csv(save_path, index=False, encoding='utf-8-sig')
    print(f"\n✅ Multi-seed summary saved to: {os.path.abspath(save_path)}")
    return df_summary


In [27]:
df_raw = pd.read_csv("rongcheng_new.csv", encoding="utf-8-sig")

multi_seed_results = run_all_variants_multi_seed(
    df_raw=df_raw,
    variants = ['No-Dilation', 'No-MoE'],
    seeds=(0, 1, 2, 3, 4),  # 5次随机种子
    tolerances=(0.10, 0.05, 0.03),
    save_prefix="./DW2/norevinandclip_multiseed"
)

multi_seed_results



####################################################################################################
🌱 Running variant: No-Dilation
####################################################################################################

[Seed = 0] =====================================
Basic Config
  Task Name:          forecast            Is Training:        1                   
  Model ID:           test                Model:              AD_MoE              

Data Loader
  Data:               BFSi                Root Path:          ./dataset/          
  Data Path:          Si.csv              Features:           MS                  
  Target:             OT                  Freq:               h                   
  Checkpoints:        ./checkpoints/      

Forecasting Task
  Seq Len:            96                  Label Len:          0                   
  Pred Len:           12                  Seasonal Patterns:  Monthly             
  Inverse:            1                   

Run 

variant                  MAE                 RMSE                 MAPE  \
0  No-Dilation  0.039745 ± 0.000302  0.050737 ± 0.000496  0.125095 ± 0.001964   
1       No-MoE  0.040026 ± 0.000545  0.051298 ± 0.001169  0.125303 ± 0.002024   

         hit_rate@±0.03        hit_rate@±0.05         hit_rate@±0.1  
0  49.898132 ± 0.302522  73.022071 ± 0.639214  96.842105 ± 0.291914  
1  49.753820 ± 0.519321  72.860781 ± 0.601460  96.782683 ± 0.154210